In [1]:
import spacy
import re
import pandas as pd
from pathlib import Path

In [2]:
df = pd.read_csv('../../dummy_data/dummy_data.csv', sep=';', index_col=0)
df['all_text'] = df.iloc[:,-3:].apply(lambda s: ' '.join([i for i in s.values if i==i]), axis=1)

In [3]:
df.head()

,MDN,NotitieID,NotitieCSN,Typenotitie,Notitiedatum,Notitietekst1,Notitietekst2,Notitietekst3,all_text
0,22325359,1428986,2,Eten/drank Ketens,2014-20-02,Nog geen twee stappen had ik in de zaak gezet ...,TEST,TEST,Nog geen twee stappen had ik in de zaak gezet ...
1,﻿60235486,1647774,2,Eten/drank Ketens,2012-31-03,Deze keten is één van de slechtste plaatsen wa...,NaN,NaN,Deze keten is één van de slechtste plaatsen wa...
2,80564281,2027552,5,Smartphones,2014-19-02,Sinds Apple in 2007 de eerste iPhone op de mar...,NaN,NaN,Sinds Apple in 2007 de eerste iPhone op de mar...
3,24105019,2267912,2,Eten/drank Ketens,2014-21-02,De laatste jaren zijn koffiebars een ‘place to...,NaN,NaN,De laatste jaren zijn koffiebars een ‘place to...
4,51373767,2453958,5,Smartphones,2014-23-02,Apple has done it again! Sinds de ontwikkeling...,NaN,NaN,Apple has done it again! Sinds de ontwikkeling...


In [6]:
def get_regex(keyword, template):
    if template == 1:
        return r'\b'+keyword+r'\b'
    elif template == 2:
        keyword = keyword.split(' ')
        return r'(\b'+keyword[0]+r'\b[^.]*?\b'+keyword[1]+r'\b|\b'+keyword[1]+r'\b[^.]*?\b'+keyword[0]+r'\b)'
    elif template == 3:
        keyword = keyword.split(' ')
        return r'\b'+keyword[0]+r'\b[^.]*?\b'+keyword[1]+r'\b[^.]*?\b'+keyword[2]+r'\b'

In [7]:
kwd = pd.read_excel('../../keywords/keywords.xlsx')
kwd['regex'] = kwd.apply(lambda row: get_regex(row.keyword, row.regex_template_id), axis=1)
kwd

,domain,keyword,regex_template_id,regex
0,FAC,lopen,1,\blopen\b
1,FAC,gelopen,1,\bgelopen\b
2,FAC,liep,1,\bliep\b
3,FAC,loopt,1,\bloopt\b
4,BER,beroep,1,\bberoep\b
5,BER,werk,1,\bwerk\b
6,BER,baan,1,\bbaan\b
7,BER,banen,1,\bbanen\b


In [8]:
reg_dict = dict()
for domain in kwd.domain.unique():
    reg_dict[domain] = '|'.join(kwd.query("domain == @domain").regex)
reg_dict

{'FAC': '\\blopen\\b|\\bgelopen\\b|\\bliep\\b|\\bloopt\\b',
 'BER': '\\bberoep\\b|\\bwerk\\b|\\bbaan\\b|\\bbanen\\b'}

In [9]:
for k, v in reg_dict.items():
    df[k] = df.all_text.str.findall(v, flags=re.IGNORECASE)

In [10]:
df

,MDN,NotitieID,NotitieCSN,Typenotitie,Notitiedatum,Notitietekst1,Notitietekst2,Notitietekst3,all_text,FAC,BER
0,22325359,1428986,2,Eten/drank Ketens,2014-20-02,Nog geen twee stappen had ik in de zaak gezet ...,TEST,TEST,Nog geen twee stappen had ik in de zaak gezet ...,[],[]
1,﻿60235486,1647774,2,Eten/drank Ketens,2012-31-03,Deze keten is één van de slechtste plaatsen wa...,NaN,NaN,Deze keten is één van de slechtste plaatsen wa...,[],[]
2,80564281,2027552,5,Smartphones,2014-19-02,Sinds Apple in 2007 de eerste iPhone op de mar...,NaN,NaN,Sinds Apple in 2007 de eerste iPhone op de mar...,[lopen],[]
3,24105019,2267912,2,Eten/drank Ketens,2014-21-02,De laatste jaren zijn koffiebars een ‘place to...,NaN,NaN,De laatste jaren zijn koffiebars een ‘place to...,[],[]
4,51373767,2453958,5,Smartphones,2014-23-02,Apple has done it again! Sinds de ontwikkeling...,NaN,NaN,Apple has done it again! Sinds de ontwikkeling...,[],[]
...,...,...,...,...,...,...,...,...,...,...,...
1295,49254354,4727838,1,Boeken,2015-20-12,Het boek ouderlijke onrust zal waarschijnlijk ...,NaN,NaN,Het boek ouderlijke onrust zal waarschijnlijk ...,[],[werk]
1296,29322831,3093347,4,Muzikanten,2015-28-12,Ik vind dat deze persoon zelfs niet in de cate...,NaN,NaN,Ik vind dat deze persoon zelfs niet in de cate...,[],[]
1297,88754703,1499889,4,Muzikanten,12/12/2015,Lana's nieuwe album gooit het over een hele ni...,NaN,NaN,Lana's nieuwe album gooit het over een hele ni...,[],[]
1298,58390637,2402111,5,Smartphones,2012-30-03,"Steve zou trots geweest zijn, de nieuwste iPho...",NaN,NaN,"Steve zou trots geweest zijn, de nieuwste iPho...",[],[]


In [11]:
df.loc[df[['FAC', 'BER']].apply(any, axis=1)][['MDN', 'NotitieID', 'NotitieCSN', 'FAC', 'BER']]

,MDN,NotitieID,NotitieCSN,FAC,BER
2,80564281,2027552,5,[lopen],[]
7,36964375,4516363,2,[loopt],[]
29,40424709,1803902,2,[loopt],[]
43,31948827,2952502,5,[loopt],[]
53,34450736,2574526,3,[loopt],[]
...,...,...,...,...,...
1266,72882086,2167824,2,[liep],[]
1282,27301037,2424486,5,[loopt],[]
1285,63965676,4781575,1,[loopt],[]
1295,49254354,4727838,1,[],[werk]


In [17]:
def text_to_conll(text, nlp):
    """
    tbd
    """
    text = text.strip()
    doc = nlp(text)
    
    # add empty line before each sentence start
    add_emptyline = lambda token: '' if token.is_sent_start else None
    tups = [(add_emptyline(token), token.text) for token in doc]

    # flatten tuples into list and drop first empty line
    tokens = [item for token in tups for item in token if item is not None][1:]
    
    # add a dummy ent_type ("O") value to each token except empty line
    tok_ents = [f"{token} O" if token != '' else '' for token in tokens]
    
    conll = '\n'.join(tok_ents)
    
    return conll

In [18]:
def row_to_conllfile(row, nlp, outdir, hospital, batch):
    """
    tbd
    """
    outfile = outdir / f"{hospital}--{row.MDN}--{row.NotitieID}--{row.NotitieCSN}--{batch}.conll"
    with open(outfile, 'w', encoding="utf-8") as f:
        f.write(text_to_conll(row.all_text, nlp))

In [23]:
import re

In [26]:
kwd1 = 'poep'
kwd2 = 'eten'
regex = re.compile(rf"\b{kwd1}\b[^.]*?\b{kwd2}\b|\b{kwd2}\b[^.]*?\b{kwd1}\b")
test = "ik wil poep heel graag met mijn handen eten, jij ook?"
test = "wat ik graag wil eten is een portie poep."
regex.search(test)

<re.Match object; span=(17, 40), match='eten is een portie poep'>

In [19]:
nlp = spacy.load("nl_core_news_sm")
outdir = Path('./test')

_ = df.tail(10).apply(lambda row: row_to_conllfile(row, nlp, outdir, 'vumc', 'batch-2021-1'), axis=1)

1290    None
1291    None
1292    None
1293    None
1294    None
1295    None
1296    None
1297    None
1298    None
1299    None
dtype: object